In [1]:
import numpy as np
import pandas as pd
from typing import Tuple
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [2]:
def generate_multivariate_time_series(
    n_steps: int = 600,
    seed: int = 42
) -> pd.DataFrame:
    """
    Generate a multivariate time series with trend, seasonality,
    autocorrelation, and cross-variable correlation.
    """
    np.random.seed(seed)
    time = np.arange(n_steps)

    seasonal = 10 * np.sin(2 * np.pi * time / 50)
    trend = 0.05 * time
    noise = np.random.normal(0, 2, n_steps)

    y1 = trend + seasonal + noise
    y2 = 0.6 * y1 + np.random.normal(0, 1.5, n_steps)
    y3 = np.roll(y1, 1) + np.random.normal(0, 1, n_steps)

    data = pd.DataFrame({
        "target": y1,
        "exog1": y2,
        "exog2": y3
    })

    return data.dropna()

In [3]:
def train_test_split_ts(
    data: pd.DataFrame,
    split_ratio: float = 0.8
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    split = int(len(data) * split_ratio)
    return data.iloc[:split], data.iloc[split:]

In [4]:
def train_sarima_model(train: pd.DataFrame):
    model = SARIMAX(
        train["target"],
        exog=train[["exog1", "exog2"]],
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 50),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    return model.fit(disp=False)

In [5]:
def forecast_with_uncertainty(
    model,
    test: pd.DataFrame,
    alpha: float = 0.10
):
    forecast_obj = model.get_forecast(
        steps=len(test),
        exog=test[["exog1", "exog2"]]
    )

    mean_forecast = forecast_obj.predicted_mean
    conf_int = forecast_obj.conf_int(alpha=alpha)

    return mean_forecast, conf_int

In [6]:
def evaluate_forecast(
    actual: pd.Series,
    predicted: pd.Series
) -> Tuple[float, float]:
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    return mae, rmse

In [7]:
def coverage_rate(
    actual: pd.Series,
    conf_int: pd.DataFrame
) -> float:
    lower = conf_int.iloc[:, 0]
    upper = conf_int.iloc[:, 1]
    return np.mean((actual >= lower) & (actual <= upper))

In [8]:
if __name__ == "__main__":
    data = generate_multivariate_time_series()
    train, test = train_test_split_ts(data)

    model = train_sarima_model(train)
    forecast, conf_int = forecast_with_uncertainty(model, test)

    mae, rmse = evaluate_forecast(test["target"], forecast)
    coverage = coverage_rate(test["target"], conf_int)

    print("MAE:", mae)
    print("RMSE:", rmse)
    print("90% Interval Coverage:", coverage)

MAE: 1.4584174475956877
RMSE: 1.794432892087881
90% Interval Coverage: 0.875
